In [1]:
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
import numpy as np
import ray
from ray.rllib import agents
from tqdm.notebook import tqdm
import random
from ray.rllib.policy.policy import Policy
from gym.spaces import Discrete, Box
from ray.rllib.agents.ppo import PPOTrainer
from functools import partial
from ray.tune.logger import pretty_print
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG

from ray.rllib.models import ModelCatalog
import uuid


import ray
from ray import tune
from ray.tune import track

import math
import gym

import gym_compete_rllib.gym_compete_to_rllib
from gym_compete_rllib.gym_compete_to_rllib import created_envs, create_env
from gym_compete_rllib.load_gym_compete_policy import nets_to_weight_array, nets_to_weights, load_weights_from_vars
from config import get_config_test_external
from train import build_trainer_config, ray_init

import pickle
from copy import deepcopy

import os
os.environ['DISPLAY'] = ':0'

In [2]:
ray_init()

{'node_ip_address': '172.20.190.40',
 'raylet_ip_address': '172.20.190.40',
 'redis_address': '172.20.190.40:6379',
 'object_store_address': '/tmp/session_2020-09-03_23-44-46_940160_11234/sockets/plasma_store',
 'raylet_socket_name': '/tmp/session_2020-09-03_23-44-46_940160_11234/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/session_2020-09-03_23-44-46_940160_11234'}

In [3]:
config_ = get_config_test_external()
config = build_trainer_config(config_)

Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents


In [4]:
config

{'env': 'multicomp',
 'env_config': {'with_video': False,
  'env_name': 'multicomp/YouShallNotPassHumans-v0'},
 'multiagent': {'policies_to_train': ['player_1', 'player_2'],
  'policies': {'player_1': (ray.rllib.policy.tf_policy_template.PPOTFPolicy,
    Box(380,),
    Box(17,),
    {'model': {'custom_model': 'GymCompetePretrainedModel',
      'custom_model_config': {'agent_id': 0,
       'env_name': 'multicomp/YouShallNotPassHumans-v0',
       'model_config': {},
       'name': 'model_0',
       'load_weights': False}},
     'framework': 'tfe'}),
   'player_2': (ray.rllib.policy.tf_policy_template.PPOTFPolicy,
    Box(380,),
    Box(17,),
    {'model': {'custom_model': 'GymCompetePretrainedModel',
      'custom_model_config': {'agent_id': 1,
       'env_name': 'multicomp/YouShallNotPassHumans-v0',
       'model_config': {},
       'name': 'model_1',
       'load_weights': True}},
     'framework': 'tfe'})},
  'policy_mapping_fn': functools.partial(<function select_policy_default at 0x

Implementation plan: change TrainOneStep, keep everything else the same!

or, do we need TrainTFMultiGPU ?
"simple_optimizer": False, by default, need multigpu



In [5]:
from ray.rllib.agents.trainer_template import build_trainer
from ray.rllib.agents import with_common_config
from collections import defaultdict
import logging
import numpy as np
import math
from typing import List

import ray
from ray.rllib.evaluation.metrics import get_learner_stats, LEARNER_STATS_KEY
from ray.rllib.evaluation.worker_set import WorkerSet
from ray.rllib.execution.common import SampleBatchType, \
    STEPS_SAMPLED_COUNTER, STEPS_TRAINED_COUNTER, LEARNER_INFO, \
    APPLY_GRADS_TIMER, COMPUTE_GRADS_TIMER, WORKER_UPDATE_TIMER, \
    LEARN_ON_BATCH_TIMER, LOAD_BATCH_TIMER, LAST_TARGET_UPDATE_TS, \
    NUM_TARGET_UPDATES, _get_global_vars, _check_sample_batch_type, \
    _get_shared_metrics
from ray.rllib.execution.multi_gpu_impl import LocalSyncParallelOptimizer
from ray.rllib.policy.policy import PolicyID
from ray.rllib.policy.sample_batch import SampleBatch, DEFAULT_POLICY_ID, \
    MultiAgentBatch
from ray.rllib.utils import try_import_tf
from ray.rllib.utils.sgd import do_minibatch_sgd, averaged

from ray.rllib.agents.ppo.ppo import DEFAULT_CONFIG as config_ppo

from ray.rllib.agents import with_common_config
from ray.rllib.agents.ppo.ppo_tf_policy import PPOTFPolicy
from ray.rllib.agents.trainer_template import build_trainer
from ray.rllib.execution.rollout_ops import ParallelRollouts, ConcatBatches, \
    StandardizeFields, SelectExperiences
from ray.rllib.execution.train_ops import TrainOneStep, TrainTFMultiGPU
from ray.rllib.execution.metric_ops import StandardMetricsReporting

tf = try_import_tf()

logger = logging.getLogger(__name__)

In [6]:
data = {}

def train_external(policies, samples, config):
    global data
    data['samples'] = samples
    data['policies'] = policies
    data['config'] = config
    return {}

In [7]:
class ExternalTrainOp:

    def __init__(self,
                 workers: WorkerSet,
                 config: dict,
                 policies: List[PolicyID] = frozenset([])):
        self.workers = workers
        self.policies = policies or workers.local_worker().policies_to_train
        self.config = config

    def __call__(self,
                 samples: SampleBatchType) -> (SampleBatchType, List[dict]):
        _check_sample_batch_type(samples)

        # Handle everything as if multiagent
        if isinstance(samples, SampleBatch):
            samples = MultiAgentBatch({
                DEFAULT_POLICY_ID: samples
            }, samples.count)

        # data: samples
            
        metrics = _get_shared_metrics()
        load_timer = metrics.timers[LOAD_BATCH_TIMER]
        learn_timer = metrics.timers[LEARN_ON_BATCH_TIMER]

        p = {k: self.workers.local_worker().get_policy(k) for k in self.policies}
        info = train_external(policies=p, samples=samples, config=self.config)
                
        load_timer.push_units_processed(samples.count)
        learn_timer.push_units_processed(samples.count)
        
        fetches = info

        metrics.counters[STEPS_TRAINED_COUNTER] += samples.count
        metrics.info[LEARNER_INFO] = fetches
        if self.workers.remote_workers():
            with metrics.timers[WORKER_UPDATE_TIMER]:
                weights = ray.put(self.workers.local_worker().get_weights(
                    self.policies))
                for e in self.workers.remote_workers():
                    e.set_weights.remote(weights, _get_global_vars())
        # Also update global vars of the local worker.
        self.workers.local_worker().set_global_vars(_get_global_vars())
        return samples, fetches


In [8]:
def execution_plan(workers, config):
    rollouts = ParallelRollouts(workers, mode="bulk_sync")

    # Collect large batches of relevant experiences & standardize.
    rollouts = rollouts.for_each(
        SelectExperiences(workers.trainable_policies()))
    rollouts = rollouts.combine(
        ConcatBatches(min_batch_size=config["train_batch_size"]))
    rollouts = rollouts.for_each(StandardizeFields(["advantages"]))

    train_op = rollouts.for_each(
        ExternalTrainOp(workers=workers,
                        config=config))

    return StandardMetricsReporting(train_op, workers, config)

In [9]:
DEFAULT_CONFIG = with_common_config({})

ExternalTrainer = build_trainer(
    name="External",
    default_config=config_ppo,#DEFAULT_CONFIG,
    default_policy=PPOTFPolicy,
    execution_plan=execution_plan)

In [10]:
trainer = ExternalTrainer(config=config)

Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
Creating agent humanoid_blocker
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Creating agent humanoid
Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
Created Scene with agents
It looks like variables {<Reference wrapping <tf.Variable 'player_1/Variable:0' shape=() dtype=int64, numpy=0>>,

In [11]:
trainer.train()

(pid=11312) Creating agent humanoid_blocker
(pid=11312) Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
(pid=11312) Creating agent humanoid
(pid=11312) Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
(pid=11312) Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
(pid=11310) Creating agent humanoid_blocker
(pid=11310) Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
(pid=11310) Creating agent humanoid
(pid=11310) Reading agent XML from: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/humanoid_body.xml
(pid=11310) Scene XML path: /home/sergei/git/chai/multiagent-competition/gym_compete/new_envs/assets/world_body.humanoid_body.humanoid_body.xml
(pid=11310) Created Scene with agents
(pid=1

{'episode_reward_max': 6.723209614283405,
 'episode_reward_min': -4.45659679104574,
 'episode_reward_mean': 5.663606208612533,
 'episode_len_mean': 161.19458128078819,
 'episodes_this_iter': 406,
 'policy_reward_min': {'player_1': -10.0, 'player_2': -14.45659679104574},
 'policy_reward_max': {'player_1': 10.0, 'player_2': 16.723209614283405},
 'policy_reward_mean': {'player_1': -8.768472906403941,
  'player_2': 14.432079115016473},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [-2.324620033004976,
   6.110690426896326,
   6.40300849959749,
   6.322113386471756,
   6.237488938189927,
   6.231690428568982,
   6.102816199669178,
   6.723209614283405,
   6.205964837506144,
   6.090901968302205,
   1.9837461224815343,
   6.259348226711154,
   6.390280697261915,
   5.895133444544626,
   6.131620572588872,
   -3.0174839981336845,
   5.949163616678561,
   6.356199912270313,
   6.465665552590508,
   6.187849105524947,
   6.082072552700993,
   6.269654702482512,
   5.969082666659233,


In [12]:
def rllib_samples_to_dict(samples):
    """Convert rllib MultiAgentBatch to a dict."""
    samples = samples.policy_batches
    samples = {x: dict(y) for x, y in samples.items()}
    return samples

def filter_dict_pickleable(d, do_print=False):
    """Keep only simple types, recurse."""
    result = {}
    allowed_types = [int, float, np.ndarray, list, dict, set, bool, str, type(None)]
    deleted_info = '_deleted'
    for x, y in d.items():
        if isinstance(y, dict):
            result[x] = filter_dict_pickleable(y, do_print=do_print)
        elif type(y) in allowed_types:
            result[x] = y
        else:
            if do_print:
                print('deleting', x, y)
            result[x] = deleted_info
    return result

def dict_get_any_value(d):
    """Return any value of a dict."""
    return list(d.values())[0]

In [13]:
data['uid'] = uuid.uuid1()
data['config'] = filter_dict_pickleable(data['config'])
data['variables_spec'] = dict_get_any_value(data['policies']).model._nets['variables_spec']
data['samples'] = rllib_samples_to_dict(data['samples'])
data['weights_dict'] = {k: nets_to_weights(v.model._nets) for k, v in data['policies'].items()}
data['policies'] = {k: nets_to_weight_array(v.model._nets) for k, v in data['policies'].items()}

In [14]:
pickle.dump(data, open('rollout.pkl', 'wb'))

In [15]:
data['weights_dict']

{'player_1': {'logstd:0': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0.]], dtype=float32),
  'retfilter/sum:0': array(1.765837e+10, dtype=float32),
  'retfilter/sumsq:0': array(2.2048401e+13, dtype=float32),
  'retfilter/count:0': array(89587710., dtype=float32),
  'obsfilter/sum:0': array([-5.61274440e+07,  6.55934680e+07,  9.46852960e+07,  7.53117360e+07,
         -2.79690625e+06,  1.50893470e+07,  6.86805950e+06, -1.15411100e+06,
         -2.77605860e+07,  7.85226200e+06, -2.66473060e+07,  2.36673600e+07,
         -1.02892980e+07, -3.81281560e+07, -6.03491750e+06, -6.26495850e+06,
         -2.10146720e+07, -4.26388720e+07, -2.46206920e+07, -5.54987640e+07,
         -9.63997680e+07,  2.84124320e+07,  1.91625420e+07, -1.12594832e+08,
          2.87220700e+07,  5.22976320e+07, -2.66275940e+07, -8.26684500e+06,
          2.75610460e+07, -3.78174075e+06,  7.20715250e+05, -3.71305400e+06,
          4.93590656e+05, -6.24160950e+06,  1.47651420e+07, -3